In [ ]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0
!pip install lyricsgenius

In [2]:
!git clone https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 613 (delta 66), reused 95 (delta 59), pack-reused 505
Receiving objects: 100% (613/613), 360.92 KiB | 7.22 MiB/s, done.
Resolving deltas: 100% (366/366), done.


In [3]:
!mkdir models/

In [4]:
!git clone https://github.com/Sushentsev/PopNet.git

Cloning into 'PopNet'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 256 (delta 69), reused 195 (delta 25), pack-reused 0
Receiving objects: 100% (256/256), 11.37 MiB | 26.11 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [5]:
%cd PopNet/data

/content/PopNet/data


In [6]:
!ls

examples	     genius_collection.py  gensongs.zip
genious_config.json  genius_examples	   lyrics_collection.py


In [ ]:
!unzip gensongs.zip

# GPT

In [8]:
import os

path = '/content/PopNet/data/gensongs/'

files = os.listdir(path)
train_files, valid_files = files[:int(len(files) * 0.8)], files[int(len(files) * 0.8):]

names = [x.split('-')[1][:-4] for x in train_files]
files = [os.path.join(path, x) for x in train_files]

train_file = open('train.txt', 'w')

for file, name in zip(files, names):
    f = open(file, "r")
    train_file.write('<startsong>\n')
    train_file.write('<songname>' + name + '\n')
    train_file.write('<songlyrics>\n')

    for line in f:
        train_file.write(line)

    train_file.write('<endsong>\n')
    f.close()

train_file.close()

names = [x.split('-')[1][:-4] for x in valid_files]
files = [os.path.join(path, x) for x in valid_files]
valid_file = open('valid.txt', 'w')

for file, name in zip(files, names):
    f = open(file, "r")
    valid_file.write('<startsong>\n')
    valid_file.write('<songname>' + name + '\n')
    valid_file.write('<songlyrics>\n')

    for line in f:
        valid_file.write(line)

    valid_file.write('<endsong>\n')
    f.close()

valid_file.close()

In [9]:
%cd ../../

/content


In [10]:
!ls

models	PopNet	ru-gpts  sample_data


In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/popnet \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=PopNet/data/train.txt \
    --do_eval \
    --eval_data_file=PopNet/data/valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

Inference


In [ ]:
import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

np.random.seed(42)
torch.manual_seed(42)

tok = GPT2Tokenizer.from_pretrained("models/popnet")
model = GPT2LMHeadModel.from_pretrained("models/popnet").cuda()

text = "<startsong>\n<songname> Дом (Home)\n<songlyrics>\n"
inpt = tok.encode(text, return_tensors="pt")

out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)
tok.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startsong>\n<songname> Дом (Home)\n<songlyrics>\nЯ хочу тебе рассказать, как я жил без тебя и не знал о том что такое больная любовь. Я хотел бы чтобы ты была рядом со мной... но только в моем сердце остались те чувства которые мы с тобой потеряли навсегда! А так хочется чтоб кто то пришел к нам жить а может быть это будет наш первый секс после долгой разлуки!!! Так хотелось забыть обо всем на свете оставив все позади нас двоих!! И вот уже который год подряд мне снится моя новая жизнь где каждый день новый мой друг живет у меня за спиной.. Мы живем словно во сне… Мне снились твои руки под моим окном когда он спит ночью просыпаясь от моих криков: "Сука!" Ты слышишь мои крики - они звучат для всех одинаково-это дом нашей мечты! Мой любимый человек теперь твой самый лучший враг номер один из тысячи людей которым можно доверять по жизни.... Но если хочешь можешь просто скажи ему об этом пусть приснится твоя первая встреча ведь она запомнит надолго даже больше чем наши сны вместе никогда 